In [14]:
import xgboost as xgb
import math
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error

import datetime as dt

In [7]:
morning_dams = pd.read_csv('../datas/questionaire_data/ques_morning_dams.csv')

sleep = pd.read_csv('../datas/fitbit_preprocessed_data/sleep.csv')
calories = pd.read_csv('../datas/fitbit_preprocessed_data/calories.csv')
floors = pd.read_csv('../datas/fitbit_preprocessed_data/floors.csv')
heart = pd.read_csv('../datas/fitbit_preprocessed_data/heart.csv')
steps = pd.read_csv('../datas/fitbit_preprocessed_data/steps.csv')
minutesFairlyActive = pd.read_csv('../datas/fitbit_preprocessed_data/minutesFairlyActive.csv')
minutesLightlyActive = pd.read_csv('../datas/fitbit_preprocessed_data/minutesLightlyActive.csv')
minutesSedentary = pd.read_csv('../datas/fitbit_preprocessed_data/minutesSedentary.csv')
minutesVeryActive = pd.read_csv('../datas/fitbit_preprocessed_data/minutesVeryActive.csv')

weather = pd.read_csv('../datas/weather_data/weather_all.csv')

In [52]:
calories['Date'] = pd.to_datetime(calories['datetime']).dt.date
calories = calories[['Date','datetime','time','user_id','level','mets','calory']]
calories_day = calories.groupby(['user_id','Date'], as_index=False).mean()

floors['Date'] = pd.to_datetime(floors['datetime']).dt.date
floors = floors[['Date','datetime', 'time', 'user_id', 'floor']]
floors_day = floors.groupby(['user_id','Date'], as_index=False).mean()

steps['Date'] = pd.to_datetime(steps['datetime']).dt.date
steps = steps[['Date','datetime', 'time', 'user_id', 'step']]
steps_day = steps.groupby(['user_id','Date'], as_index=False).mean()

heart['Date'] = pd.to_datetime(heart['datetime']).dt.date
heart = heart[['Date','datetime', 'time', 'user_id', 'heart']]
heart_day = heart.groupby(['user_id','Date'], as_index=False).mean()

sleep['Date'] = pd.to_datetime(sleep['datetime']).dt.date
sleep = sleep[['Date','datetime', 'dateTime', 'user_id', 'sleep']]
sleep_day = sleep.groupby(['user_id','Date'], as_index=False).mean()

minutesFairlyActive['Date'] = pd.to_datetime(minutesFairlyActive['datetime']).dt.date
minutesFairlyActive = minutesFairlyActive[['Date','datetime', 'time', 'user_id', 'minutesFairlyActive']]
minutesFairlyActive_day = minutesFairlyActive.groupby(['user_id','Date'], as_index=False).mean()

minutesLightlyActive['Date'] = pd.to_datetime(minutesLightlyActive['datetime']).dt.date
minutesLightlyActive = minutesLightlyActive[['Date','datetime', 'time', 'user_id', 'minutesLightlyActive']]
minutesLightlyActive_day = minutesLightlyActive.groupby(['user_id','Date'], as_index=False).mean()

minutesSedentary['Date'] = pd.to_datetime(minutesSedentary['datetime']).dt.date
minutesSedentary = minutesSedentary[['Date','datetime', 'time', 'user_id', 'minutesSedentary']]
minutesSedentary_day = minutesSedentary.groupby(['user_id','Date'], as_index=False).mean()

minutesVeryActive['Date'] = pd.to_datetime(minutesVeryActive['datetime']).dt.date
minutesVeryActive = minutesVeryActive[['Date','datetime', 'time', 'user_id', 'minutesVeryActive']]
minutesVeryActive_day = minutesVeryActive.groupby(['user_id','Date'], as_index=False).mean()

<ipython-input-52-dcc3ced2a5cf>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minutesFairlyActive['Date'] = pd.to_datetime(minutesFairlyActive['datetime']).dt.date
<ipython-input-52-dcc3ced2a5cf>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minutesLightlyActive['Date'] = pd.to_datetime(minutesLightlyActive['datetime']).dt.date
<ipython-input-52-dcc3ced2a5cf>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [53]:
fitbit_merge = pd.merge(calories_day, floors_day, on=['user_id','Date'])
fitbit_merge = pd.merge(fitbit_merge, heart_day,on=['user_id','Date'])
fitbit_merge = pd.merge(fitbit_merge, sleep_day,on=['user_id','Date'])
fitbit_merge = pd.merge(fitbit_merge, steps_day,on=['user_id','Date'])
fitbit_merge = pd.merge(fitbit_merge, minutesFairlyActive_day,on=['user_id','Date'])
fitbit_merge = pd.merge(fitbit_merge, minutesLightlyActive_day,on=['user_id','Date'])
fitbit_merge = pd.merge(fitbit_merge, minutesSedentary_day,on=['user_id','Date'])
fitbit_merge = pd.merge(fitbit_merge, minutesVeryActive_day,on=['user_id','Date'])
# fitbit_merge.to_csv('../datas/fitbit_merge_day.csv')

In [73]:
# morning_dams = morning_dams.drop('Unnamed: 0',axis=1)
# morning_dams = morning_dams.drop('Unnamed: 0.1',axis=1)
weather = weather.drop('Unnamed: 0',axis=1).drop('Unnamed: 0.1',axis=1)

In [3]:
def main():
    # データセットを読み込む
    X = df[['']]   # 説明変数
    y = df['']     # 目的変数
    # データセットを学習用と検証用に分割する
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        test_size=0.3,
                                                        shuffle=True,
                                                        random_state=42,
                                                        stratify=y)
    # XGBoost が扱うデータセットの形式に直す
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    # 学習用のパラメータ
    xgb_params = {
        # 分類の種類
        'objective': 'multi:softmax',
        # 評価指標
        'eval_metric': 'logloss',
    }
    # モデルを学習する
#     evals = [(dtrain, 'train'), (dtest, 'eval')]
#     evals_result = {}
    bst = xgb.train(xgb_params,
                    dtrain,
                    num_boost_round=100,  # 学習ラウンド数は適当
                    )
    # 検証用データが各クラスに分類される確率を計算する
    y_pred_proba = bst.predict(dtest)
    # しきい値 0.5 で 0, 1 に丸める
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    # 精度 (Accuracy) を検証する
    acc = accuracy_score(y_test, y_pred)
    print('Accuracy:', acc)
    recall = recall_score(y_test, y_pred)
    print('recall:', recall)
    prec = precision_score(y_test, y_pred)
    print('precision',prec)
    f1 = f1_score(y_test, y_pred)
    print('f1-score',f1)
    #confusion matrix
    mat = confusion_matrix(y_test,y_pred)
    sns.heatmap(mat,square=True,annot=True,cbar=False,fmt='d',cmap='RdPu')
    plt.xlabel('predicted class')
    plt.ylabel('true value')
if __name__ == '__main__':
    main()

NameError: name 'df' is not defined